## This is a solution for the Loan Prediction Practice Problem contest:   
https://datahack.analyticsvidhya.com/contest/practice-problem-loan-prediction-iii/
started:  
Wed May 25 2016 20:31:00 GMT+0200 (CEST)  
closed: Mon May 01 2017 20:29:00 GMT+0200 (CEST)  
Prizes: Knowledge and Learning

In [760]:
import sklearn
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [761]:
# This function replaces variables in a dataframe acording to a map determined by the dictionary
# passed to the second argument:

def coding(col, codeDict):
    colCoded = pd.Series(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

# Replaces a column of a dataframe to a new  column determined by the dictionary passed to its third argument.
def to_numeric(df, colname, dictionary, new_colname) :
    df.loc[:,new_colname] = coding(df.loc[:,colname], dictionary)
    del df[colname]
    #df.drop(colname, axis=1, inplace=True)

In [762]:
# This function replaces variables in a dataframe acording to a map determined by the dictionary
# passed to the second argument:

def coding(col, codeDict):
    colCoded = pd.Series(col, copy=True)
    for key, value in codeDict.items():
        colCoded.replace(key, value, inplace=True)
    return colCoded

# Replaces a column of a dataframe to a new  column determined by the dictionary passed to its third argument.
def to_numeric(df, colname, dictionary, new_colname) :
    df.loc[:,new_colname] = coding(df.loc[:,colname], dictionary)
    del df[colname]
    #df.drop(colname, axis=1, inplace=True)

In [763]:
raw_data = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')
raw_test = pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')

In [764]:
raw_data.shape

(614, 13)

In [766]:
raw_data.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [767]:
raw_data['istrain']=1
raw_test['istrain']=0
#1
target=raw_data['Loan_Status']
del raw_data['Loan_Status']

In [768]:
frames = [raw_data, raw_test]
alldata = pd.concat(frames)

In [769]:
for col in alldata.columns:
    if alldata[col].isnull().sum()>0:
        print(col, alldata[col].isnull().sum())

('Gender', 24)
('Married', 3)
('Dependents', 25)
('Self_Employed', 55)
('LoanAmount', 27)
('Loan_Amount_Term', 20)
('Credit_History', 79)


In [770]:

#imputing with "others":

alldata["Married"].fillna("others", inplace=True)

In [771]:
print alldata["Gender"].unique()

['Male' 'Female' nan]


In [772]:
#imputing with "others":

alldata["Gender"].fillna("others", inplace=True)

In [773]:
print alldata["Dependents"].unique()

['0' '1' '2' '3+' nan]


In [774]:
#imputing with "others":

alldata["Dependents"].fillna("others", inplace=True)

In [775]:
dict_Dependents={'0':1,'1':2,'2':3,
        '3+':4, 'others':-1}


to_numeric(alldata, "Dependents", dict_Dependents, "new_Dep")

In [776]:
print alldata["Education"].unique()

['Graduate' 'Not Graduate']


In [777]:
#imputing with "others":

#alldata["Dependents"].fillna("others", inplace=True)

In [778]:
print alldata['Self_Employed'].unique()

['No' 'Yes' nan]


In [779]:
#imputing with "others":

alldata['Self_Employed'].fillna("others", inplace=True)

In [780]:
#imputing with "others":

alldata['LoanAmount'].fillna(-1, inplace=True)

In [781]:
print alldata['Loan_Amount_Term'].unique()

[ 360.  120.  240.   nan  180.   60.  300.  480.   36.   84.   12.  350.
    6.]


In [782]:
#imputing with "others":

alldata['Loan_Amount_Term'].fillna(-1, inplace=True)
print alldata['Loan_Amount_Term'].unique()

[ 360.  120.  240.   -1.  180.   60.  300.  480.   36.   84.   12.  350.
    6.]


In [783]:
print alldata['Credit_History'].unique()

[  1.   0.  nan]


In [784]:
#imputing with "others":

alldata['Credit_History'].fillna(-1, inplace=True)
print alldata['Credit_History'].unique()

[ 1.  0. -1.]


# New Features

In [785]:
np.sum(alldata['CoapplicantIncome']!=0)

552

In [786]:
alldata["new1"]=alldata["LoanAmount"]/alldata["Loan_Amount_Term"]
alldata["new2"]=alldata["CoapplicantIncome"]+alldata["ApplicantIncome"]
alldata["new4"]=alldata["new1"]/alldata["new2"]
alldata["new2"]=np.log(alldata["new2"])

alldata["new3"]=(alldata["CoapplicantIncome"]+alldata["ApplicantIncome"])/alldata['Loan_Amount_Term']

In [787]:
alldata=pd.get_dummies(alldata, drop_first=True, columns=['Gender', 'Married', 'Education',
       'Self_Employed', 
        'Property_Area'])

In [788]:
#help(pd.get_dummies)

In [789]:
train_df=alldata[alldata["istrain"]==1]
test_df=alldata[alldata["istrain"]==0]


In [790]:
train_df.ix[:,'Loan_Status']=target

In [791]:
h2o.init(max_mem_size = "2G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 hours 7 mins
H2O cluster version:,3.10.0.9
H2O cluster version age:,5 months and 6 days !!!
H2O cluster name:,H2O_from_python_badicst_n4264c
H2O cluster total nodes:,1
H2O cluster free memory:,1.612 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [792]:
train_df.columns
train_df=h2o.H2OFrame(train_df)
test_df=h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Validation:

In [793]:
trainl, validl, testl = train_df.split_frame([0.6, 0.2], seed=1235)

In [794]:
X = trainl.col_names[:-1]     #last column is the response variable 
y = trainl.col_names[-1] 
from enum import Enum
rf_v1 = H2ORandomForestEstimator(
    model_id="rf_covType_v1",
    balance_classes=True,
    max_depth=3,
    ntrees=1000,
    stopping_rounds=2,
    stopping_tolerance=0.0000001,
    score_each_iteration=True,
    seed=1000000,
    nfolds=5)
rf_v1.train(X, y, training_frame=trainl, validation_frame=validl)
final_rf_predictions = rf_v1.predict(testl[:-1])
pred=final_rf_predictions.as_data_frame(use_pandas=True)
pred=np.array(pred["predict"])
true_labels=np.array(testl["Loan_Status"].as_data_frame(use_pandas=True))
metrics.accuracy_score(true_labels, pred, normalize=True)


drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


0.74380165289256195

In [795]:
final_rf_predictions_val = rf_v1.predict(validl[:-1])
predv=final_rf_predictions_val.as_data_frame(use_pandas=True)
predv=np.array(predv["predict"])
true_labelsv=np.array(validl["Loan_Status"].as_data_frame(use_pandas=True))
from sklearn.metrics import accuracy_score
accuracy_score(true_labelsv, predv)

drf prediction progress: |████████████████████████████████████████████████| 100%


0.73109243697478987

In [796]:
rf_v1.cross_validation_metrics_summary()

Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.7715284,0.0242406,0.7848101,0.8333333,0.7460318,0.7464789,0.7469879
auc,0.7430612,0.0323796,0.8033333,0.7786132,0.6722075,0.7192691,0.7418831
err,0.2284716,0.0242406,0.2151899,0.1666667,0.2539683,0.2535211,0.2530121
err_count,17.0,1.8439089,17.0,13.0,16.0,18.0,21.0
f0point5,0.7993364,0.0190848,0.8121019,0.8456973,0.7859532,0.7692308,0.7836990
f1,0.8488566,0.0214724,0.8571429,0.8976378,0.8545455,0.8085107,0.8264463
f2,0.9052496,0.0271707,0.9074733,0.9563758,0.936255,0.8520179,0.8741259
lift_top_group,1.4664127,0.0783844,1.462963,1.3684211,1.3404255,1.6511627,1.5090909
logloss,0.5611854,0.0288787,0.5537322,0.5004358,0.5500679,0.6265777,0.5751133
max_per_class_error,0.6429524,0.1311385,0.56,0.6190476,1.0,0.4642857,0.5714286


In [797]:
rf_v1.model_performance(trainl)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.170453334143
RMSE: 0.412859944948
LogLoss: 0.520606073037
Mean Per-Class Error: 0.217657574153
AUC: 0.847540386653
Gini: 0.695080773305
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.608545518629: 


,N,Y,Error,Rate
N,67.0,51.0,0.4322,(51.0/118.0)
Y,15.0,241.0,0.0586,(15.0/256.0)
Total,82.0,292.0,0.1765,(66.0/374.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.6085455,0.8795620,71.0
max f2,0.5901724,0.9370370,83.0
max f0point5,0.6688245,0.8624620,55.0
max accuracy,0.6398012,0.8262032,61.0
max precision,0.8201244,1.0,0.0
max recall,0.4907591,1.0,107.0
max specificity,0.8201244,1.0,0.0
max absolute_mcc,0.6398012,0.5848836,61.0
max min_per_class_accuracy,0.7412053,0.765625,32.0
max mean_per_class_accuracy,0.6688245,0.7823424,55.0


Gains/Lift Table: Avg response rate: 68.45 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0213904,0.8026389,1.4609375,1.4609375,1.0,1.0,0.03125,0.03125,46.09375,46.09375
,2,0.0213904,0.7991030,0.0,1.4609375,0.0,1.0,0.0,0.03125,-100.0,46.09375
,3,0.0374332,0.7946967,1.4609375,1.4609375,1.0,1.0,0.0234375,0.0546875,46.09375,46.09375
,4,0.0401070,0.7876209,1.4609375,1.4609375,1.0,1.0,0.0039062,0.0585938,46.09375,46.09375
,5,0.0668449,0.7847977,1.4609375,1.4609375,1.0,1.0,0.0390625,0.0976562,46.09375,46.09375
,6,0.1203209,0.7753758,1.2417969,1.3635417,0.85,0.9333333,0.0664062,0.1640625,24.1796875,36.3541667
,7,0.1604278,0.7672625,1.3635417,1.3635417,0.9333333,0.9333333,0.0546875,0.21875,36.3541667,36.3541667
,8,0.2192513,0.7670857,1.328125,1.3540396,0.9090909,0.9268293,0.078125,0.296875,32.8125,35.4039634
,9,0.3021390,0.7592017,1.3666835,1.3575083,0.9354839,0.9292035,0.1132812,0.4101562,36.6683468,35.7508296
,10,0.4786096,0.7563036,1.2617188,1.3221892,0.8636364,0.9050279,0.2226562,0.6328125,26.171875,32.2189246


# Prediction

In [798]:
X = train_df.col_names[:-1]     #last column is the response variable 
y = train_df.col_names[-1] 
rf_v1.train(X, y, training_frame=train_df)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [799]:
prediction_default  = rf_v1.predict(test_df)

prediction_default.names=["Loan_Status", "dumy1","dumy2"]

drf prediction progress: |████████████████████████████████████████████████| 100%


In [800]:
submitfile_h2o = test_df[['Loan_ID']]



In [801]:
submitfile_h2o=submitfile_h2o.cbind(prediction_default[["Loan_Status"]])



In [802]:
submitfile_h2o=submitfile_h2o.as_data_frame(use_pandas=True)

In [803]:
submitfile_h2o.to_csv("submissionh2o.csv", index=False)

In [804]:
#h2o.shutdown()